In [1]:
# 导入必要的库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from imblearn.over_sampling import SMOTE

# 读取处理后的数据
data = pd.read_csv('iBite_table_processed.csv')

# 提取空间数据 (仅使用经纬度)
spatial_features = ['latitude', 'longitude']

# 提取体型数据 (头部和身体的尺寸数据)
morphological_features = ['head.w', 'head.h', 'head.l', 'th.w', 'body.l', 'wing.l', 'head_w_body_l_ratio', 'head_volume', 'wing_body_ratio']

# 提取目标变量
y = data['iBite']

# 标准化特征
scaler = StandardScaler()

# 仅使用空间数据
X_spatial = scaler.fit_transform(data[spatial_features])

# 结合空间数据和体型数据
X_combined = scaler.fit_transform(data[spatial_features + morphological_features])

# 使用 SMOTE 平衡数据
sm = SMOTE(random_state=42)
X_spatial_resampled, y_spatial_resampled = sm.fit_resample(X_spatial, y)
X_combined_resampled, y_combined_resampled = sm.fit_resample(X_combined, y)

# 定义模型
models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(random_state=42),
    'LightGBM': lgb.LGBMRegressor(random_state=42),
    'CatBoost': cb.CatBoostRegressor(random_state=42, verbose=0),
    'SVR': SVR(),
    'GPR': GaussianProcessRegressor(kernel=C(1.0) * RBF(length_scale=1.0))
}

# 定义模型参数
params = {
    'RandomForest': {'n_estimators': [100, 200], 'max_depth': [10, 20], 'max_features': ['sqrt', 'log2']},
    'XGBoost': {'n_estimators': [100, 200], 'max_depth': [3, 5], 'learning_rate': [0.1, 0.05], 'subsample': [0.8, 1.0]},
    'LightGBM': {'n_estimators': [100, 200], 'max_depth': [10, 20], 'learning_rate': [0.1, 0.05], 'num_leaves': [31, 50]},
    'CatBoost': {'iterations': [100, 200], 'depth': [6, 8], 'learning_rate': [0.1, 0.05]},
    'SVR': {'kernel': ['rbf'], 'C': [1, 10], 'gamma': ['scale', 'auto']},
    'GPR': {'alpha': [1e-10, 1e-5]}
}

# 函数：模型训练和评估
def train_and_evaluate(X_train, y_train, X_test, y_test, model_name, param_grid):
    model = models[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    # 最佳模型
    best_model = grid_search.best_estimator_
    print(f"最佳参数 ({model_name}): {grid_search.best_params_}")
    
    # 在测试集上评估
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} 测试集 MSE: {mse:.2f}, R²: {r2:.2f}")
    
    return mse, r2

# 空间数据 vs 体型+空间数据的对比
def compare_models(X_resampled, y_resampled, label):
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

    results = []
    for model_name in models.keys():
        print(f"训练和评估模型 ({label}): {model_name}")
        mse, r2 = train_and_evaluate(X_train, y_train, X_test, y_test, model_name, params[model_name])
        results.append({'Model': model_name, 'MSE': mse, 'R²': r2})
    
    return pd.DataFrame(results)

# 使用仅空间数据进行训练
print("\n--- 仅空间数据 ---")
spatial_results = compare_models(X_spatial_resampled, y_spatial_resampled, "仅空间数据")

# 使用体型+空间数据进行训练
print("\n--- 空间+体型数据 ---")
combined_results = compare_models(X_combined_resampled, y_combined_resampled, "体型+空间数据")

# 比较结果
print("\n--- 结果对比 ---")
print("仅使用空间数据的结果：")
print(spatial_results)
print("结合体型和空间数据的结果：")
print(combined_results)



--- 仅空间数据 ---
训练和评估模型 (仅空间数据): RandomForest
最佳参数 (RandomForest): {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 200}
RandomForest 测试集 MSE: 246427.35, R²: 0.50
训练和评估模型 (仅空间数据): XGBoost
最佳参数 (XGBoost): {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
XGBoost 测试集 MSE: 242952.43, R²: 0.50
训练和评估模型 (仅空间数据): LightGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001610 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 167
[LightGBM] [Info] Number of data points in the train set: 722, number of used features: 2
[LightGBM] [Info] Start training from score 1367.671745
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light

/opt/anaconda3/envs/comp7811/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/envs/comp7811/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/envs/comp7811/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/envs/comp7811/lib/

最佳参数 (GPR): {'alpha': 1e-05}
GPR 测试集 MSE: 282386.47, R²: 0.42

--- 空间+体型数据 ---
训练和评估模型 (体型+空间数据): RandomForest
最佳参数 (RandomForest): {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 200}
RandomForest 测试集 MSE: 272923.50, R²: 0.44
训练和评估模型 (体型+空间数据): XGBoost
最佳参数 (XGBoost): {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
XGBoost 测试集 MSE: 275447.42, R²: 0.44
训练和评估模型 (体型+空间数据): LightGBM

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

/opt/anaconda3/envs/comp7811/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/envs/comp7811/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/envs/comp7811/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/envs/comp7811/lib/


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

/opt/anaconda3/envs/comp7811/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

/opt/anaconda3/envs/comp7811/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/opt/anaconda3/envs/comp7811/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/envs/comp7811/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warn


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 